In [1]:
from entities import *
from constants import TEAMS_DATA, STAT_EFFECTS, POSSESSIONS_STD_DEV
from simulations import *

import pickle
import numpy as np
import pandas as pd

In [2]:
with open(TEAMS_DATA, 'rb') as f:
    teams = pickle.load(f)

with open(STAT_EFFECTS, 'rb') as f:
    stats = pickle.load(f)

In [3]:
az = teams['Arizona']
af = teams['Air Force']

In [4]:
from IPython.display import clear_output
from tqdm import tqdm, trange

In [ ]:
def parse_game_log(log):
    
    team_1_log = log[0]
    team_2_log = log[1]

    team_1_players = list(team_1_log.keys())
    team_2_players = list(team_2_log.keys())
    players = [team_1_players, team_2_players]

    event_log = {}
    for player in (team_1_players + team_2_players):
        event_log[player] = {}

    for i, team_log in enumerate([team_1_log, team_2_log]):
        for player in players[i]:
            for event, count  in team_log[player][2].items():
                if not event_log.get(player).get(event):
                    event_log[player][event] = count
                else:
                    event_log[player][event] += count

    return event_log


from collections import Counter
import pandas as pd
from tqdm import trange


def simulate_n_games(team_1, team_2, n: int, summary=False, viz=False):
    print(f'Simulating Game: {team_1.name} vs. {team_2.name}')

    # 1. Initialize Score Tracking
    # Using lists to store every score for distribution analysis
    game_results = {
        team_1.name: [],
        team_2.name: []
    }
    
    team_1_players = list(team_1.df.index)
    team_2_players = list(team_2.df.index)
    
    total_stats_log = {}
    player_games_count = Counter() # Specifically for "Games Played"

    for i in trange(n):
        game = simulate_game(team_1, team_2)
        
        # 2. Append Scores for statistical analysis
        # We match the score to the correct team name list
        if game['Winner'] == team_1.name:
            game_results[team_1.name].append(game['Winner Score'])
            game_results[team_2.name].append(game['Loser Score'])
        else:
            game_results[team_2.name].append(game['Winner Score'])
            game_results[team_1.name].append(game['Loser Score'])

        # 3. Parse and Update Stats
        game_log = parse_game_log(game['Game Log'])

        for player, stats in game_log.items():
            # Track participation
            player_games_count[player] += 1
            
            # Update cumulative stats
            if player not in total_stats_log:
                total_stats_log[player] = Counter(stats)
            else:
                total_stats_log[player].update(stats)

    # 4. Final Processing
    df = pd.DataFrame(total_stats_log).T.fillna(0)
    
    # Add the Games Played column
    # .map(player_games_count) matches the player index to the Counter value
    df['Games Played'] = df.index.map(player_games_count)
    df['Team'] = [team_1.name if p in team_1_players else team_2.name for p in list(df.index)]

    print(f'{team_1.name} vs. {team_2.name} Complete!')
    
    # We return the DataFrame AND the raw scores for your distribution analysis
    return df, game_results

In [48]:
az_af = simulate_n_games(az, af, 100)

Simulating Game: Arizona vs. Air Force


100%|██████████| 100/100 [00:05<00:00, 18.18it/s]

Arizona vs. Air Force Complete!


In [49]:
az_af[0]

,2P,3P,BLK,STL,TOV,ORB,DRB,PF,Games Played,Team
Anthony Dell'Orso,236.882824,166.616927,29.811581,124.214922,178.869487,29.811581,178.869487,154.026503,71,Arizona
Brayden Burries,282.554433,109.930564,37.583099,159.728169,197.311268,37.583099,244.290141,206.707043,67,Arizona
Dwayne Aristode,64.914571,130.845541,0.000000,21.174987,31.762481,66.171836,119.109305,66.171836,38,Arizona
Ivan Kharchenkov,181.707724,79.835415,34.795096,135.314262,112.117532,81.188557,270.628524,135.314262,55,Arizona
Jaden Bradley,404.366140,85.767206,0.000000,156.921131,156.921131,34.326497,245.189268,205.958985,70,Arizona
Ethan Greenberg,76.552640,173.553905,23.853748,143.122486,172.939671,41.744059,280.281536,172.939671,85,Air Force
Kam Sanders,192.568299,138.691465,10.679254,128.151042,213.585070,53.396268,368.434246,240.283204,76,Air Force
Kolby Gilles,100.337932,14.298824,42.810851,42.810851,200.675864,115.054162,115.054162,288.973244,38,Air Force
Lucas Hobin,141.649179,282.757099,23.791590,53.531077,142.749540,41.635282,261.707490,226.020105,85,Air Force
Wesley Celichowski,304.515847,24.297652,97.190608,35.342039,145.785912,75.101834,291.571825,340.167129,63,Air Force
